In [25]:
#Es necesario importar las depencendias necesarias
import pandas as pd
from datetime import date
from datetime import datetime


In [26]:
# Importamos el dataset de trabajo
df = pd.read_csv('./dataset/movies_dataset.csv', delimiter=',')
# Se elimina las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage
df = df.drop(columns = ['video','imdb_id','adult','original_title','poster_path','homepage']) 
# df.head()
print(" El dataset tiene", df.shape[0],"filas. ")

 El dataset tiene 45466 filas. 


C:\Users\HP\AppData\Local\Temp\ipykernel_9696\369045348.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./dataset/movies_dataset.csv', delimiter=',')


In [27]:
# Metodo para detectar los datos duplicados
# me sirve para ver si existen registros duplicados
df.duplicated().sum()
# Se averigua cuantos registros ("diferentes") duplicados hay.

13

In [28]:
# Se eliminan esos registros duplicados:

df = df.drop_duplicates()
# datafrane sin duplicados
df.shape

print(" El dataset queda con ", df.shape[0],"filas. ")

 El dataset queda con  45453 filas. 


In [29]:
# Se averigua cuantos valores nulos se tiene por columna:
df.isnull().sum()

print("Los campos que poseen nulos son: \n",df.isnull().sum())

Los campos que poseen nulos son: 
 belongs_to_collection    40959
budget                       0
genres                       0
id                           0
original_language           11
overview                   954
popularity                   5
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25045
title                        6
vote_average                 6
vote_count                   6
dtype: int64


In [30]:
# COLUMNA revenue
# Imputamos con el valor 0 los campos nulos de la con una revenue - recaudacion de la pelicula

df['revenue'].fillna(0, inplace=True)

df.isnull().sum()

belongs_to_collection    40959
budget                       0
genres                       0
id                           0
original_language           11
overview                   954
popularity                   5
production_companies         3
production_countries         3
release_date                87
revenue                      0
runtime                    263
spoken_languages             6
status                      87
tagline                  25045
title                        6
vote_average                 6
vote_count                   6
dtype: int64

In [31]:
# Se averigua el tipo de dato de cada columna y se decide si es el tipo de dato correcto
# Asimismo al corregir el tipo de dato, se analizan los datos cargados. budge es el presupuesto de la pelicula pasar a float
# Pero primero remplazamos otros valores.
df['budget'].replace('/ff9qCepilowshEtG2GYWwzt2bs4.jpg', 0, inplace = True)
df['budget'].replace('/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg', 0, inplace = True)
df['budget'].replace('/zaSf5OG7V8X8gqFvly88zDdRm46.jpg', 0, inplace = True)

df['budget'] = df['budget'].astype(float)

df['budget'].dtype

dtype('float64')

In [32]:
# VEEEEER belongs_to_collection
# VERRRR genres

df['id'].duplicated().sum()
# Se averigua cuantos registros ("diferentes") duplicados hay. ver estooooo

17

In [33]:
# Los valores nulos del campo release date (fecha de estreno) deben eliminarse. 
# Se decide eliminar pues no serviran para las consultas en la API posteriormente.

df['release_date'].replace('1',None, inplace = True)
df['release_date'].replace('12',None, inplace = True)
df['release_date'].replace('22',None, inplace = True)
df.dropna(subset='release_date',inplace=True )

df.isnull().sum()


belongs_to_collection    40875
budget                       0
genres                       0
id                           0
original_language           11
overview                   941
popularity                   0
production_companies         0
production_countries         0
release_date                 0
revenue                      0
runtime                    246
spoken_languages             0
status                      80
tagline                  24969
title                        0
vote_average                 0
vote_count                   0
dtype: int64

In [34]:
print(" El dataset tiene", df.shape[0],"filas. ")

 El dataset tiene 45363 filas. 


In [35]:
df['release_date'].sort_values(ascending=True)

34940    1874-12-09
34937    1878-06-14
41602    1883-11-19
34933    1887-08-18
34934    1888-01-01
            ...    
44535    2018-04-04
38130    2018-04-25
30402    2018-11-07
38885    2018-12-31
26559    2020-12-16
Name: release_date, Length: 45363, dtype: object

In [43]:
# De haber fechas, deberán tener el formato AAAA-mm-dd, 
# además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.
# Se crea tambien las columnas release_month y release_day para ser utilizadas en los metodos de la api.

df['release_date'].to_string(index=False)


'1995-10-30\\n1995-12-15\\n1995-12-22\\n1995-12-22\\...\n1995-10-30\\n1995-12-15\\n1995-12-22\\n1995-12-22\\...\n1995-10-30\\n1995-12-15\\n1995-12-22\\n1995-12-22\\...\n1995-10-30\\n1995-12-15\\n1995-12-22\\n1995-12-22\\...\n1995-10-30\\n1995-12-15\\n1995-12-22\\n1995-12-22\\...\n1995-10-30\\n1995-12-15\\n1995-12-22\\n1995-12-22\\...\n1995-10-30\\n1995-12-15\\n1995-12-22\\n1995-12-22\\...\n1995-10-30\\n1995-12-15\\n1995-12-22\\n1995-12-22\\...\n1995-10-30\\n1995-12-15\\n1995-12-22\\n1995-12-22\\...\n1995-10-30\\n1995-12-15\\n1995-12-22\\n1995-12-22\\...\n1995-10-30\\n1995-12-15\\n1995-12-22\\n1995-12-22\\...\n1995-10-30\\n1995-12-15\\n1995-12-22\\n1995-12-22\\...\n1995-10-30\\n1995-12-15\\n1995-12-22\\n1995-12-22\\...\n1995-10-30\\n1995-12-15\\n1995-12-22\\n1995-12-22\\...\n1995-10-30\\n1995-12-15\\n1995-12-22\\n1995-12-22\\...\n1995-10-30\\n1995-12-15\\n1995-12-22\\n1995-12-22\\...\n1995-10-30\\n1995-12-15\\n1995-12-22\\n1995-12-22\\...\n1995-10-30\\n1995-12-15\\n1995-12-22\\n1995-12-

In [37]:
df['title']  = [line for line in df['title']  if line.strip()]
df['title'] = df['title'].astype(str)
df['title'].sort_values(ascending=True)



18757                                !Women Art Revolution
30961                                  #1 Cheerleader Camp
36153                                              #Horror
23501                                         #chicagoGirl
28042                                  $1,000 on the Black
                               ...                        
22346                                       هیچ کجا هیچ کس
38463    ‘Rameau’s Nephew’ by Diderot (Thanx to Dennis ...
8882                                       ’Round Midnight
39033                      …And the Fifth Horseman Is Fear
41473                                             ファンタスティポ
Name: title, Length: 45363, dtype: object

In [38]:
df.to_csv('../movies/dataset/dataset_limpio.csv', index=False)